In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [2]:
!mkdir -p /content/data
!tar -xvf /content/drive/MyDrive/pothole-detection/train.tar  -C /content/data
!tar -xvf /content/drive/MyDrive/pothole-detection/test1.tar  -C /content/data


Streaming output truncated to the last 5000 lines.
train/Japan/annotations/xmls/Japan_005633.xml
train/Japan/annotations/xmls/Japan_013040.xml
train/Japan/annotations/xmls/Japan_001374.xml
train/Japan/annotations/xmls/Japan_003791.xml
train/Japan/annotations/xmls/Japan_006601.xml
train/Japan/annotations/xmls/Japan_007041.xml
train/Japan/annotations/xmls/Japan_003242.xml
train/Japan/annotations/xmls/Japan_002794.xml
train/Japan/annotations/xmls/Japan_008264.xml
train/Japan/annotations/xmls/Japan_003415.xml
train/Japan/annotations/xmls/Japan_011187.xml
train/Japan/annotations/xmls/Japan_009508.xml
train/Japan/annotations/xmls/Japan_005601.xml
train/Japan/annotations/xmls/Japan_004580.xml
train/Japan/annotations/xmls/Japan_012353.xml
train/Japan/annotations/xmls/Japan_009857.xml
train/Japan/annotations/xmls/Japan_011774.xml
train/Japan/annotations/xmls/Japan_005531.xml
train/Japan/annotations/xmls/Japan_012534.xml
train/Japan/annotations/xmls/Japan_010129.xml
train/Japan/annotations/xmls/

In [3]:
!rm -rf /content/data/train/Czech /content/data/train/Japan
!rm -rf /content/data/test1/Czech /content/data/test1/Japan


In [4]:
!mkdir -p /content/dataset/images/train
!mkdir -p /content/dataset/images/val
!mkdir -p /content/dataset/images/test
!mkdir -p /content/dataset/labels/train
!mkdir -p /content/dataset/labels/val


In [5]:
!cp /content/data/train/India/images/* /content/dataset/images/train/
!cp /content/data/test1/India/images/* /content/dataset/images/test/


In [6]:
import xml.etree.ElementTree as ET
from pathlib import Path

XML_DIR = Path("/content/data/train/India/annotations/xmls")
OUT_DIR = Path("/content/dataset/labels/train")
OUT_DIR.mkdir(parents=True, exist_ok=True)

IMG_W = IMG_H = 720

for xml in XML_DIR.glob("*.xml"):
    root = ET.parse(xml).getroot()
    rows = []
    for obj in root.findall("object"):
        if obj.find("name").text != "D40":
            continue
        b = obj.find("bndbox")
        xmin, ymin = float(b.find("xmin").text), float(b.find("ymin").text)
        xmax, ymax = float(b.find("xmax").text), float(b.find("ymax").text)

        xc = ((xmin + xmax) / 2) / IMG_W
        yc = ((ymin + ymax) / 2) / IMG_H
        w  = (xmax - xmin) / IMG_W
        h  = (ymax - ymin) / IMG_H

        rows.append(f"0 {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}")

    if rows:
        (OUT_DIR / f"{xml.stem}.txt").write_text("\n".join(rows))


In [7]:
!ls /content/dataset/images/train | tail -n 200 | xargs -I {} mv /content/dataset/images/train/{} /content/dataset/images/val/


In [8]:
!for f in /content/dataset/images/val/*; do \
  mv /content/dataset/labels/train/$(basename ${f%.*}).txt /content/dataset/labels/val/ 2>/dev/null; \
done


In [9]:
%%writefile pothole.yaml
path: /content/dataset
train: images/train
val: images/val

names:
  0: pothole


Writing pothole.yaml


In [12]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.train(
    data="pothole.yaml",
    epochs=30,
    imgsz=576,
    batch=32,
    workers=8,
    cache=True,
    device=0
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.245 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=pothole.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=576, int8=False, iou=0.7, keras=False, kobj=1.0, line

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a8792b117c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 